In [52]:
!pip install keras
!pip install tensorflow

In [45]:
import numpy as np
import pandas as pd
import re
import pickle
import os
import nltk
from nltk.corpus import stopwords
from textblob import Word
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.activations import softmax # Import softmax

In [46]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # For multilingual WordNet support
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
pip install -U textblob

In [48]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [49]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
file_path = '/content/test.ft.txt'  # Update this with the correct file path
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Process the data to extract labels and reviews
data = []
for line in lines:
    # Match the pattern: __label__ followed by a number and review text
    match = re.match(r"(__label__\d)\s(.+)", line.strip())
    if match:
        label = match.group(1)  # Extract the label
        review = match.group(2)  # Extract the review text
        data.append([label, review])

# Convert the data into a DataFrame
df = pd.DataFrame(data, columns=['Label', 'Review'])
df = df[:150000]


In [53]:
df.head()

,Label,Review
0,__label__2,Great CD: My lovely Pat has one of the GREAT v...
1,__label__2,One of the best game music soundtracks - for a...
2,__label__1,Batteries died within a year ...: I bought thi...
3,__label__2,"works fine, but Maha Energy is better: Check o..."
4,__label__2,Great for the non-audiophile: Reviewed quite a...


In [54]:
def cleaning(df, stop_words):
    # Convert to lowercase and split
    df['Review'] = df['Review'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

    # Replacing special characters
    df['Review'] = df['Review'].str.replace('[^a-zA-Z\s]', '', regex=True)  # Keep only letters and spaces

    # Replacing digits/numbers
    df['Review'] = df['Review'].str.replace(r'\d+', '', regex=True)  # Remove all digits

    # Removing stop words
    df['Review'] = df['Review'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))

    # Lemmatization
    df['Review'] = df['Review'].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))

    return df

# Apply cleaning function
df = cleaning(df, stop_words)


In [55]:
tokenizer = Tokenizer(num_words=10000, split=' ')
tokenizer.fit_on_texts(df['Review'].values)

In [56]:

file_path = 'my_sentiment_model.pkl'  # Replace with your pickle file path
file_size = os.path.getsize(file_path)

print(f"The size of '{file_path}' is: {file_size} bytes")

The size of 'my_sentiment_model.pkl' is: 602968601 bytes


In [57]:
!md5sum my_sentiment_model.pkl

96f77109f6b1c3717766e35d2a2b15e3  my_sentiment_model.pkl


In [58]:
with open('my_sentiment_model.pkl', 'rb') as file:
  model = pickle.load(file)

In [61]:
def modifyReviews(reviews_array):
    """
    Preprocesses an array of input reviews to match the format used during training.

    Args:
        reviews_array (list): A list of reviews to be preprocessed.

    Returns:
        numpy.ndarray: The preprocessed reviews as a padded sequence.
    """
    import re # Import the re module for regular expression operations

    processed_reviews = []
    for review in reviews_array:
        # Convert to lowercase and split
        review = ' '.join(review.lower() for review in review.split())

        # Replacing special characters using re.sub()
        review = re.sub('[^a-zA-Z\s]', '', review)  # Keep only letters and spaces

        # Replacing digits/numbers using re.sub()
        review = re.sub(r'\d+', '', review)  # Remove all digits

        # Removing stop words
        review = ' '.join(review for review in review.split() if review not in stop_words)

        # Lemmatization
        review = ' '.join([Word(word).lemmatize() for word in review.split()])

        processed_reviews.append(review)

    # Convert to sequence and pad
    X = tokenizer.texts_to_sequences(processed_reviews)
    X = pad_sequences(X, maxlen=103)

    return X

In [73]:
new_reviews = ["This is a great product!", "I love this item.", "Not so good.", "I hate this thing", "its soo good that i will recommend this to my friends", "worst thing"]

# Preprocess the reviews array using modifyReviews function
processed_reviews = modifyReviews(new_reviews)

# Make predictions using your model
predictions = model.predict(processed_reviews)
# Apply softmax to the predictions
predictions = softmax(predictions).numpy() # Apply softmax and convert to numpy array

print(predictions)
# Classify and print sentiment
threshold = 0.5  # Adjust the threshold as needed
for i, prediction in enumerate(predictions):
    if prediction[1] >= threshold:  # Assuming index 0 represents positive sentiment
        print(f"Review {i + 1}: Good sentiment")
    else:
        print(f"Review {i + 1}: Not good sentiment")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
[[0.26894197 0.73105794]
 [0.26894757 0.73105234]
 [0.27091366 0.72908634]
 [0.70596665 0.2940333 ]
 [0.2727263  0.72727376]
 [0.28034478 0.7196552 ]]
Review 1: Good sentiment
Review 2: Good sentiment
Review 3: Good sentiment
Review 4: Not good sentiment
Review 5: Good sentiment
Review 6: Good sentiment


In [74]:
# Rank reviews based on positive sentiment probability
ranked_reviews = sorted(
    [(i, prediction[1]) for i, prediction in enumerate(predictions)],
    key=lambda item: item[1],
    reverse=True,  # Sort in descending order (highest probability first)
)

# Print ranked reviews
print("Ranked Reviews (from most good to least):")
for rank, (index, probability) in enumerate(ranked_reviews):
    print(f"Rank {rank + 1}: Review {index + 1} (Probability: {probability:.2f})")

Ranked Reviews (from most good to least):
Rank 1: Review 1 (Probability: 0.73)
Rank 2: Review 2 (Probability: 0.73)
Rank 3: Review 3 (Probability: 0.73)
Rank 4: Review 5 (Probability: 0.73)
Rank 5: Review 6 (Probability: 0.72)
Rank 6: Review 4 (Probability: 0.29)
